Cómo Funciona
Modelo LLM:

Utilizamos un modelo de Hugging Face (distilbert-base-uncased) configurado para tareas de clasificación de texto.
Este modelo devuelve un puntaje de similitud entre el texto procesado (consulta + texto objetivo).
División en Palabras:

La consulta se divide en palabras para que SHAP pueda evaluar la importancia de cada una.
predict_text:

Genera texto filtrado basado en las palabras activas y calcula un puntaje de similitud con el modelo LLM.
Referencia SHAP:

Una matriz aleatoria de referencia ayuda a SHAP a generar explicaciones al comparar diferentes combinaciones de palabras.
Visualización:

Se utiliza shap.force_plot para mostrar cómo cada palabra contribuye al puntaje de similitud.

In [1]:
import shap
import numpy as np
from transformers import pipeline


In [ ]:
import shap
from sentence_transformers import SentenceTransformer
import numpy as np

# Modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Consulta del usuario
query = "¿Cómo ayuda la cúrcuma a la salud?"

# Dividir la consulta en palabras
words = query.split()

# Función para generar embeddings de cada palabra
def word_to_vector(word):
    return model.encode([word])[0]

# Generar el vector completo para la consulta
query_vector = model.encode([query])[0]

# Definir una función de predicción que combine palabras
def predict_text(words_to_keep):
    """Genera el vector para la consulta solo con palabras seleccionadas."""
    # Convertir el array booleano en una frase filtrada
    filtered_query = " ".join([words[i] for i in range(len(words)) if words_to_keep[i] > 0.5])
    
    # Si no hay palabras seleccionadas, retorna un vector nulo
    if not filtered_query.strip():
        return np.zeros_like(query_vector)
    
    # Retornar el embedding del texto filtrado
    return model.encode([filtered_query])[0]

# Crear una referencia (matriz) para SHAP (conjunto de palabras completo)
reference = np.eye(len(words))  # Cada fila representa una palabra activa

# Crear el explicador SHAP
explainer = shap.KernelExplainer(predict_text, reference)

# Calcular los valores SHAP para la consulta completa
shap_values = explainer.shap_values(np.ones(len(words)))

# Visualizar los valores SHAP
shap.force_plot(explainer.expected_value, shap_values, words)
